In [56]:
import pandas as pd
import numpy as np
import ipyvolume.pylab as p3
import pymatgen as pmg

import lammps

In [60]:
a = 4.2
alpha = 90
symbols = ['Mg', 'O']
positions = [(0, 0, 0), (0.5, 0.5, 0.5)]
supercell = (3, 3, 3)
lattice = pmg.Lattice.from_parameters(a, a, a, alpha, alpha, alpha)
structure = pmg.Structure.from_spacegroup(225, lattice, symbols, positions) * supercell

lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(structure)
elements = [_.symbol for _ in elements]

charges = {
    'Mg': 2,
    'O': -2
}
potential = {
    ('Mg', 'Mg'): [1309362.2766468062, 0.104, 0.0],
    ('Mg', 'O'): [9892.357, 0.20199, 0.0],
    ('O', 'O'): [2145.7345, 0.3, 30.2222]
}

lmp.command('velocity all create 3.0 100')
lmp.command('kspace_style pppm 1.0e-4')
lmp.command('pair_style buck/coul/long 10.0')

# set buck
for p, coeffs in potential.items():
    ij = ' '.join([str(_ + 1) for _ in sorted([elements.index(p[0]), elements.index(p[1])])])
    coeffs = ' '.join([str(_) for _ in coeffs])
    lmp.command('pair_coeff {} {}'.format(ij, coeffs))

# set charge
for e, charge in charges.items():
    lmp.command('set atom {} charge {}'.format(elements.index(e) + 1, charge))

lmp.command('fix 1 all nve')

'fix'

In [61]:
positions = []
atom_types = []
N = 100
for i in range(N):
    lmp.run(5)
    positions.append(lmp.system.positions.copy())
    atom_types.append(lmp.system.types.copy())
    
positions = np.array(positions).reshape(N, lmp.system.total, 3)
x = positions[:, :, 0]
y = positions[:, :, 1]
z = positions[:, :, 2]

size = np.array(atom_types).reshape(N, lmp.system.total).astype(float)

color_array  = np.array([
    (103, 169, 207),
    (239, 138, 98)
]) / 256
color = color_array[np.array(atom_types).reshape(N, lmp.system.total)  - 1]

In [64]:
p3.figure()
s = p3.scatter(x, y, z, color=color, size=size, marker="sphere")
p3.xyzlim(1, supercell[0] * a - 1)
p3.animation_control(s) # shows controls for animation controls
p3.show()